In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
GPU = torch.cuda.is_available()
print(GPU)
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

True


In [2]:
GPU = False
METAPRECISION = False

In [3]:
from __future__ import print_function
from __future__ import division
import os
import sys
import torch
import tqdm
import pdb
import numpy as np
import platform
import hashlib
import pytorch_transformer
import re
import argparse
import tensorflow as tf
from tensorflow.python import pywrap_tensorflow
import torch.nn.functional as F
#from torch.utils.tensorboard import SummaryWriter
from transformProtein import transformProtein
from ProteinDataset_uid import ProteinDataset
from torch.utils.data import Dataset, DataLoader
import pickle
import time
import matplotlib.pyplot as plt
torch.cuda.empty_cache()
load_model_path = 'ckpt/' # just the folder itself

seq_length = 511
embedding_dim = 1280
num_layers = 36
vocab_loc = 'mapping_files/vocab.txt'

use_py3 = platform.python_version()[0] == '3'
vocab = open(vocab_loc).readlines() if not use_py3 else open(vocab_loc, encoding='utf-8').read().split('\n')[:-1]
vocab = list(map(lambda x: x.split(' ')[0], vocab))
vocab_size = len(vocab)
print('-----vocab size',vocab_size,'------')

class TiedEmbeddingSoftmax(torch.nn.Module):

    def __init__(self, vocab_size=vocab_size, embedding_size=embedding_dim, **kwargs):
        super(TiedEmbeddingSoftmax, self).__init__()
        self.w = torch.nn.Parameter(torch.normal(0., 1e-2, size=(vocab_size, embedding_size)))
        self.b = torch.nn.Parameter(torch.zeros(vocab_size))

    def forward(self, inputs, embed=True):
        with autocast(enabled=METAPRECISION):
            if embed:
                return torch.nn.functional.embedding(inputs, self.w)
            else:
                return torch.tensordot(inputs, self.w.t(), 1) + self.b

class CTRLmodel(torch.nn.Module):
    def __init__(self):
        super(CTRLmodel,self).__init__()
        self.tied_embedding_softmax = TiedEmbeddingSoftmax()
        self.encoder = pytorch_transformer.Encoder()

    def forward(self, inputs):
        with autocast(enabled=METAPRECISION):
            x = self.tied_embedding_softmax(inputs, embed=True)
            x = self.encoder(x)
            x = self.tied_embedding_softmax(x, embed=False)
        return x

    def loadCheckpoint(self, model_path, num_layers):
        if os.path.exists(model_path):
            print('Found PyTorch checkpoint at ', model_path)
            print('Loading instead of converting from TensorFlow')
            checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
            self.tied_embedding_softmax.load_state_dict({
                'w': checkpoint.pop('tied_embedding_softmax.w', None),
                'b': checkpoint.pop('tied_embedding_softmax.b', None)
            })
            self.encoder.load_state_dict({key.replace("encoder.", ""): value for key, value in checkpoint.items()})
        else:
            print('Could not find PyTorch checkpoint')
            sys.exit()

model = CTRLmodel()
print('model initialized')

curr_model_path = load_model_path+'pretrain_progen_full.pth'
reader = model.loadCheckpoint(model_path=curr_model_path, num_layers = num_layers)
print('previous checkpoint loaded')

'''
def compute_scaling_factor(model):
    # Initialize the scaling factor to a small value
    scale_factor = 1e-8
    # Iterate over all the model's parameters to find the maximum absolute value
    for param in model.parameters():
        if param.requires_grad:
            max_abs_value = torch.max(torch.abs(param.data))
            scale_factor = max(scale_factor, max_abs_value)
    return scale_factor

def scale_weights(model, scale_factor=128.0):
    # Scale all the model's parameters by the given factor
    for param in model.parameters():
        if param.requires_grad:
            param.data /= scale_factor
'''
            
if METAPRECISION:
    #scale_factor = compute_scaling_factor(model)
    #print("scaling factor: ", scale_factor)
    #scale_weights(model, scale_factor)
    model = model.to(torch.float16)
if GPU:
    model = model.cuda()
    print('previous checkpoint loaded in GPU')

optimizer = torch.optim.Adam(model.parameters()) #lr, betas

model.eval()

with open(os.path.join('mapping_files/','taxa_to_lineage.p'),'rb') as handle:
    taxa_to_lineage = pickle.load(handle)
with open('mapping_files/taxa_to_ctrl_idx.p','rb') as handle:
    taxa_to_ctrl_idx = pickle.load(handle)
with open('mapping_files/kw_to_ctrl_idx.p','rb') as handle:
    kw_to_ctrl_idx = pickle.load(handle)
with open('mapping_files/aa_to_ctrl_idx.p','rb') as handle:
    aa_to_ctrl_idx = pickle.load(handle)
    
with open('mapping_files/kw_to_name.p2','rb') as handle:
    kw_to_name = pickle.load(handle)
#with open('mapping_files/taxid_to_name.p2','rb') as handle:
#    taxid_to_name = pickle.load(handle)
    
def flipdict(my_map):
    return {v: k for k, v in my_map.items()}
ctrl_idx_to_aa = flipdict(aa_to_ctrl_idx)
ctrl_idx_to_kw = flipdict(kw_to_ctrl_idx)
ctrl_idx_to_taxa = flipdict(taxa_to_ctrl_idx)

def predict_fn(inputs):
    with torch.no_grad():
        inputs = torch.tensor(inputs)
        if GPU:
            inputs = inputs.cuda()            
        output = model(inputs)
        output = output[:,:,-26:-1] # remove non-AA token logits
        return output

-----vocab size 129407 ------
MODEL SIZE: 
1280
model initialized
Found PyTorch checkpoint at  ckpt/pretrain_progen_full.pth
Loading instead of converting from TensorFlow
previous checkpoint loaded


In [4]:

# paths to the saved .p files
query = "PF01832"
random_selection_file = os.path.join("data", "random_selection_" + query + ".p")
data_file = os.path.join("data", "filtered_data_" + query + ".p")

# Reload random_selection from the .p file
random_selection = False
with open(random_selection_file, "rb") as file:
    random_selection = pickle.load(file)

# Reload filtered_data from the .p file
#filtered_data = False
#with open(data_file, "rb") as file:
#    filtered_data = pickle.load(file)

# Print or use the reloaded datasets as needed
print("Random Selection:")
for entry in random_selection:
    print(entry["sequence"])
    break

#print("---")

#print("Filtered Data:")
#for entry in filtered_data:
#    print(entry["sequence"])'''


Random Selection:
{'metadata': {'accession': 'A0A015KT78', 'id': 'A0A015KT78_9BACL', 'source_organism': {'taxId': '1380763', 'scientificName': 'Paenibacillus darwinianus', 'fullName': 'Paenibacillus darwinianus'}, 'name': 'SH3b domain-containing protein', 'description': None, 'length': 351, 'sequence': 'MYSKKNRSLQFVCVLSAIFILILTTGMLVATTENEPSAANAYAASLLTAVEGRKPEKSQQKPVPPPVSAAAALRKPTEPVGTLALSSVSAAASADSVAKSLDTPARPLPPSPAAKRYKVTAYHLNVRANADLSSDIIRVAEKGAVLDVVSATGNGWFKLAGEGYVSGKYVQPASLTGKVSGQVAVLSATMPPGARTEVKPEPAIAIHKLGNVQKPSSAVNSRSGLTESHIADLLTNTALAEQGLEQAILEIENEYGINAYFTIAVMKLESGNGKSRLARNKNNLFGLMARSGALSFETKGDSIRKFGQLIAKNYVGQGYATIEKVAGKYCPVNPGWPGLVKGIMKSDYRKL', 'proteome': 'UP000052954', 'gene': 'BG52_02005', 'go_terms': [{'identifier': 'GO:0004040', 'name': 'amidase activity', 'category': {'code': 'F', 'name': 'molecular_function'}}], 'protein_evidence': 4, 'source_database': 'unreviewed', 'is_fragment': False, 'ida_accession': '896ea4d3ff813f75fe0f53a020683f6c500d13fc', 'counters': {'domain_arc

<p>To evaluate the generated sequence using teacher forcing:

1. Generate the Sequence: The transformer model with teacher forcing generates the complete sequence. 
At each time step, we provide the true input sequence (+ 1 versus previous step), as input to the model.

2. Calculate Metrics: Accuracy, soft accuracy (based on BLOSUM matrix), perplexity. 

</p>

In [5]:

def teacher_forcing_generation(input_sequence, penalty, topk):
    key_len = 0
    res = ""
    tokens_prob = []
    true_tokens_index_in_prob = []
    for i in range(1, len(input_sequence)):
        iteration_input_prefix = input_sequence[:i]
        seed_seq = [aa_to_ctrl_idx[ii] for ii in iteration_input_prefix]
        generate_num = len(iteration_input_prefix) + 1 # how many tokens to generate
        padded_text = seed_seq + [0] * (generate_num - len(seed_seq))
        tokens_generated = np.tile(padded_text, (1,1))
        for token in range(len(seed_seq)-1, generate_num-1):
            prompt_logits = predict_fn(tokens_generated[:, :seq_length]).squeeze()
            _token = token if token < seq_length else -1
            prompt_logits = prompt_logits.cpu().detach().numpy()
            #print(tokens_generated[:, :seq_length])
            #print(prompt_logits)
            if penalty>0:
                penalized_so_far = set()
                # variable token_flag for first amminoacids (to count them if they are less that 4)
                if token >= key_len + 3:
                    token_flag = 3  
                elif token - key_len - 3 <= 0:
                    token_flag = 0
                else:
                    token_flag = token
                for _ in range(token-token_flag,token+1):
                    generated_token = tokens_generated[0][_] - (vocab_size-26) # added
                    if generated_token in penalized_so_far:
                        continue
                    penalized_so_far.add(generated_token)
                    prompt_logits[_token][generated_token] /= penalty  
            # compute probabilities from logits
            prompt_probs = np.exp(prompt_logits[_token])
            prompt_probs = prompt_probs / sum(prompt_probs)
            pruned_list = np.argsort(prompt_probs)[::-1]
            codice_aa_della_true_sequence = seed_seq[_token]
            idx_true = codice_aa_della_true_sequence - vocab_size + 26
            true_tokens_index_in_prob.append([idx_true])
            tokens_prob.append([prompt_probs.tolist()])

            if topk==1:
                idx = pruned_list[0]
            else:
                pruned_list = pruned_list[:topk]
                chosen_idx = torch.distributions.categorical.Categorical(logits=torch.tensor(np.expand_dims(prompt_logits[_token][pruned_list],0))).sample().numpy()[0]
                idx = pruned_list[chosen_idx]
            # assign the token for generation
            idx += (vocab_size-26) # added to convert 0 AA to original ctrl idx
            tokens_generated[0][token+1] = idx
        tokens_generated_so_far = tokens_generated[0].squeeze()[:token+2]
        tokens_generated_so_far = tokens_generated_so_far[(tokens_generated_so_far>=(vocab_size-26)) & (tokens_generated_so_far<(vocab_size-1))]
        tokens_generated_so_far = ''.join([ctrl_idx_to_aa[c] for c in tokens_generated_so_far])
        query = tokens_generated_so_far[len(seed_seq):]
        res += query
    return res, tokens_prob, true_tokens_index_in_prob


In [6]:

import pickle
# print("on data: ", query)
penalty = 1.2
topk = 3
predicted = []
true_tokens_index_in_probs_all = []
tokens_probs_all = []
true_value = []
for entry in random_selection:
    input_seq = entry['sequence']['metadata']['sequence']
    # FOR TESTIG:
    # input_seq = input_seq[0:10]
    
    try:
        res, tokens_prob, true_tokens_index_in_prob = teacher_forcing_generation(input_seq, penalty, topk)
    except:
        print("ALERT: skipped for error the entry: "entry)
        continue
    # print("Input: ", input_seq)
    # print("Res: ", res)
    true_value.append(input_seq)
    true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    tokens_probs_all.append(tokens_prob)
    predicted.append(input_seq[0] + res)

    # FOR TESTIG:
    break
    
    

MYSKKNRSLQFVCVLSAIFILILTTGMLVATTENEPSAANAYAASLLTAVEGRKPEKSQQKPVPPPVSAAAALRKPTEPVGTLALSSVSAAASADSVAKSLDTPARPLPPSPAAKRYKVTAYHLNVRANADLSSDIIRVAEKGAVLDVVSATGNGWFKLAGEGYVSGKYVQPASLTGKVSGQVAVLSATMPPGARTEVKPEPAIAIHKLGNVQKPSSAVNSRSGLTESHIADLLTNTALAEQGLEQAILEIENEYGINAYFTIAVMKLESGNGKSRLARNKNNLFGLMARSGALSFETKGDSIRKFGQLIAKNYVGQGYATIEKVAGKYCPVNPGWPGLVKGIMKSDYRKL
[[129392      0]]
[[16.721256    4.6247196  14.177379   15.835678   15.933856   15.410613
  15.924175   14.927818   15.941516   16.624912   16.243252   14.921511
  16.154205   -2.7780662  16.048454   15.515584   16.264513   16.8818
  16.621109    3.611012   15.748315   13.955129   11.013475   14.82486
   3.9392157 ]
 [ 9.868706   -0.8429748   9.01112     9.038809   10.461087   10.203462
   9.862633    9.002217   11.034259   11.157314   10.650659   10.27542
   9.675399   -3.2261677   9.66336    10.31101     9.81564     9.857248
   9.793587    2.8387613  10.432395    7.7890425   5.439209   10.147632
  -0.11435249]]
[[129392 129404      0]]
[[

[[129392 129404 129398 129390 129390 129393 129397      0]]
[[16.721256   4.6247215 14.177381  15.835682  15.933858  15.410614
  15.924173  14.927822  15.941518  16.624914  16.243252  14.921512
  16.154207  -2.7780662 16.048454  15.515584  16.264517  16.8818
  16.62111    3.6110115 15.748317  13.9551325 11.013477  14.824862
   3.9392166]
 [15.475255   4.52244   14.121091  15.184022  15.301435  15.164516
  15.277004  14.543221  15.423201  15.896282  15.879809  14.167602
  15.135907  -3.1762893 15.119238  15.277516  15.703851  15.685127
  15.4104595  3.762671  15.270759  13.711152  10.255001  14.870971
   3.9246888]
 [15.586563   4.373681  14.183061  15.210579  15.392402  15.485942
  15.405516  14.606964  15.806498  15.758893  16.058847  14.598175
  15.276433  -2.4528198 15.40727   15.06795   15.613364  15.955939
  15.526267   4.636257  15.596604  13.886576  10.138287  15.070516
   3.4852614]
 [15.327766   4.5525846 13.793723  15.208122  15.900897  15.265369
  15.120367  14.477387  16.10

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386      0]]
[[16.721256   4.6247215 14.177381  15.835682  15.933858  15.410614
  15.924173  14.927822  15.941518  16.624914  16.243252  14.921512
  16.154207  -2.7780662 16.048454  15.515584  16.264517  16.8818
  16.62111    3.6110115 15.748317  13.9551325 11.013477  14.824862
   3.9392166]
 [15.475255   4.52244   14.121091  15.184022  15.301435  15.164516
  15.277004  14.543221  15.423201  15.896282  15.879809  14.167602
  15.135907  -3.1762893 15.119238  15.277516  15.703851  15.685127
  15.4104595  3.762671  15.270759  13.711152  10.255001  14.870971
   3.9246888]
 [15.586563   4.373681  14.183061  15.210579  15.392402  15.485942
  15.405516  14.606964  15.806498  15.758893  16.058847  14.598175
  15.276433  -2.4528198 15.40727   15.06795   15.613364  15.955939
  15.526267   4.636257  15.596604  13.886576  10.138287  15.070516
   3.4852614]
 [15.327766   4.5525846 13.793723  15.208122  15.900897  15.265369

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401      0]]
[[16.721256   4.6247215 14.177381  15.835682  15.933858  15.410614
  15.924173  14.927822  15.941518  16.624914  16.243252  14.921512
  16.154207  -2.7780662 16.048454  15.515584  16.264517  16.8818
  16.62111    3.6110115 15.748317  13.9551325 11.013477  14.824862
   3.9392166]
 [15.475255   4.52244   14.121091  15.184022  15.301435  15.164516
  15.277004  14.543221  15.423201  15.896282  15.879809  14.167602
  15.135907  -3.1762893 15.119238  15.277516  15.703851  15.685127
  15.4104595  3.762671  15.270759  13.711152  10.255001  14.870971
   3.9246888]
 [15.586563   4.373681  14.183061  15.210579  15.392402  15.485942
  15.405516  14.606964  15.806498  15.758893  16.058847  14.598175
  15.276433  -2.4528198 15.40727   15.06795   15.613364  15.955939
  15.526267   4.636257  15.596604  13.886576  10.138287  15.070516
   3.4852614]
 [15.327766   4.5525846 13.793723  15.208122 

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398      0]]
[[16.721256   4.6247215 14.177381  15.835682  15.933858  15.410614
  15.924173  14.927822  15.941518  16.624914  16.243252  14.921512
  16.154207  -2.7780662 16.048454  15.515584  16.264517  16.8818
  16.62111    3.6110115 15.748317  13.9551325 11.013477  14.824862
   3.9392166]
 [15.475255   4.52244   14.121091  15.184022  15.301435  15.164516
  15.277004  14.543221  15.423201  15.896282  15.879809  14.167602
  15.135907  -3.1762893 15.119238  15.277516  15.703851  15.685127
  15.4104595  3.762671  15.270759  13.711152  10.255001  14.870971
   3.9246888]
 [15.586563   4.373681  14.183061  15.210579  15.392402  15.485942
  15.405516  14.606964  15.806498  15.758893  16.058847  14.598175
  15.276433  -2.4528198 15.40727   15.06795   15.613364  15.955939
  15.526267   4.636257  15.596604  13.886576  10.138287  15.070516
   3.4852614]
 [15.327766   4.5525846 13.7937

...

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398 129381 129389 129386 129389
  129391 129389 129391 129399 129399 129387 129392 129391 129401 129381
  129399 129399 129385 129393 129385 129395 129398 129381 129381 129393
  129381 129404 129381 129381 129398 129391 129391 129399 129381 129401
  129385 129387 129397 129390 129395 129385 129390 129398 129396 129396
  129390 129395 129401 129395 129395 129395 129401 129398 129381 129381
  129381 129381 129391 129397 129390 129395 129399 129385 129395 129401
  129387 129399 129391 129381 129391 129398 129398 129401 129398 129381
  129381 129381 129398 129381 129384 129398 129401 129381 129390 129398
  129391 129384 129399 129395 129381 129397 129395 129391 129395 129395
  129398 129395 129381 129381 129390 129397 129404 129390 129401 129399
  129381 129404 129388 129391 129393 129401 129397 129381 129393 129381
  129384 129391 129398 129398 129384 129389 129389 129397 129401

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398 129381 129389 129386 129389
  129391 129389 129391 129399 129399 129387 129392 129391 129401 129381
  129399 129399 129385 129393 129385 129395 129398 129381 129381 129393
  129381 129404 129381 129381 129398 129391 129391 129399 129381 129401
  129385 129387 129397 129390 129395 129385 129390 129398 129396 129396
  129390 129395 129401 129395 129395 129395 129401 129398 129381 129381
  129381 129381 129391 129397 129390 129395 129399 129385 129395 129401
  129387 129399 129391 129381 129391 129398 129398 129401 129398 129381
  129381 129381 129398 129381 129384 129398 129401 129381 129390 129398
  129391 129384 129399 129395 129381 129397 129395 129391 129395 129395
  129398 129395 129381 129381 129390 129397 129404 129390 129401 129399
  129381 129404 129388 129391 129393 129401 129397 129381 129393 129381
  129384 129391 129398 129398 129384 129389 129389 129397 129401

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398 129381 129389 129386 129389
  129391 129389 129391 129399 129399 129387 129392 129391 129401 129381
  129399 129399 129385 129393 129385 129395 129398 129381 129381 129393
  129381 129404 129381 129381 129398 129391 129391 129399 129381 129401
  129385 129387 129397 129390 129395 129385 129390 129398 129396 129396
  129390 129395 129401 129395 129395 129395 129401 129398 129381 129381
  129381 129381 129391 129397 129390 129395 129399 129385 129395 129401
  129387 129399 129391 129381 129391 129398 129398 129401 129398 129381
  129381 129381 129398 129381 129384 129398 129401 129381 129390 129398
  129391 129384 129399 129395 129381 129397 129395 129391 129395 129395
  129398 129395 129381 129381 129390 129397 129404 129390 129401 129399
  129381 129404 129388 129391 129393 129401 129397 129381 129393 129381
  129384 129391 129398 129398 129384 129389 129389 129397 129401

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398 129381 129389 129386 129389
  129391 129389 129391 129399 129399 129387 129392 129391 129401 129381
  129399 129399 129385 129393 129385 129395 129398 129381 129381 129393
  129381 129404 129381 129381 129398 129391 129391 129399 129381 129401
  129385 129387 129397 129390 129395 129385 129390 129398 129396 129396
  129390 129395 129401 129395 129395 129395 129401 129398 129381 129381
  129381 129381 129391 129397 129390 129395 129399 129385 129395 129401
  129387 129399 129391 129381 129391 129398 129398 129401 129398 129381
  129381 129381 129398 129381 129384 129398 129401 129381 129390 129398
  129391 129384 129399 129395 129381 129397 129395 129391 129395 129395
  129398 129395 129381 129381 129390 129397 129404 129390 129401 129399
  129381 129404 129388 129391 129393 129401 129397 129381 129393 129381
  129384 129391 129398 129398 129384 129389 129389 129397 129401

[[129392 129404 129398 129390 129390 129393 129397 129398 129391 129396
  129386 129401 129383 129401 129391 129398 129381 129389 129386 129389
  129391 129389 129391 129399 129399 129387 129392 129391 129401 129381
  129399 129399 129385 129393 129385 129395 129398 129381 129381 129393
  129381 129404 129381 129381 129398 129391 129391 129399 129381 129401
  129385 129387 129397 129390 129395 129385 129390 129398 129396 129396
  129390 129395 129401 129395 129395 129395 129401 129398 129381 129381
  129381 129381 129391 129397 129390 129395 129399 129385 129395 129401
  129387 129399 129391 129381 129391 129398 129398 129401 129398 129381
  129381 129381 129398 129381 129384 129398 129401 129381 129390 129398
  129391 129384 129399 129395 129381 129397 129395 129391 129395 129395
  129398 129395 129381 129381 129390 129397 129404 129390 129401 129399
  129381 129404 129388 129391 129393 129401 129397 129381 129393 129381
  129384 129391 129398 129398 129384 129389 129389 129397 129401

In [23]:
# Create a directory named "ID_test_data" in the current working directory if it doesn't exist
data_dir = "ID_test_data"
os.makedirs(data_dir, exist_ok=True)

# TODO, salvare tokens_prob e true_tokens_index_in_prob
tokens_probs_data_file = os.path.join(data_dir, "tokens_probs_data_" + query + ".p")
with open(tokens_probs_data_file, "wb") as file:
    pickle.dump(tokens_probs_all, file)
    
true_tokens_index_data_file = os.path.join(data_dir, "true_tokens_index_data_" + query + ".p")
with open(true_tokens_index_data_file, "wb") as file:
    pickle.dump(true_tokens_index_in_probs_all, file)

# Save ID_test_data as a .p file
predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
with open(predicted_data_file, "wb") as file:
    pickle.dump(predicted, file)
    
# Save ID_test_data as a .p file
true_data_file = os.path.join(data_dir, "true_data_" + query + ".p")
with open(true_data_file, "wb") as file:
    pickle.dump(true_value, file)
torch.cuda.empty_cache()

In [24]:
print(input_seq[0] + res)

MKLLLELSLKIKILLFIFFLSSAGSVFASYGFSKSSLIVVESVTTLREESPSKSLLAVLLAEAATKAEAKETSSPQASSVSALPPPGTPLTPTPVPLPTTPPTPVSLSSASAAVTSSSPGIGSPSSDAGGTLPRAELRAALSLTLTSIKAGTGLVRFRLTPITTARAWYVALGGGKPAPLETLLTSKKGVGSATSLVVAALSAVVTTGVSAVTALAAATTLGPAAPPSVLVLSQKKAGGPGGSVTASVVRVQVAGGISLRRLPSGGREAAGKLPAAAVGIILGGVGIPDDGGTIVPNDLNFIFYVHLLLAASDGLGASITSVVLAGGRDTSLLGASVLSSTETASDDAGVA


In [25]:
print(input_seq)

MYSKKNRSLQFVCVLSAIFILILTTGMLVATTENEPSAANAYAASLLTAVEGRKPEKSQQKPVPPPVSAAAALRKPTEPVGTLALSSVSAAASADSVAKSLDTPARPLPPSPAAKRYKVTAYHLNVRANADLSSDIIRVAEKGAVLDVVSATGNGWFKLAGEGYVSGKYVQPASLTGKVSGQVAVLSATMPPGARTEVKPEPAIAIHKLGNVQKPSSAVNSRSGLTESHIADLLTNTALAEQGLEQAILEIENEYGINAYFTIAVMKLESGNGKSRLARNKNNLFGLMARSGALSFETKGDSIRKFGQLIAKNYVGQGYATIEKVAGKYCPVNPGWPGLVKGIMKSDYRKL


In [26]:
len(input_seq)

351

<h3>Evaluations</h3>

In [85]:
# Da implementare compute_perplexity
from nltk import ngrams
from sklearn.metrics import accuracy_score
import math
import blosum as bl
import torch
from torcheval.metrics.text import Perplexity
import decimal
# desired precision for printing float values
decimal.getcontext().prec = 18
#  BLOSUM62 is the matrix built using sequences with less than 62% similarity 
# (sequences with ≥ 62% identity were clustered)

def compute_hard_accuracy(true_string, predicted_string):
    # percentage of predictions that exactly match the ground truth labels
    total_characters = len(true_string)
    assert total_characters == len(predicted_string)
    # Convert strings to lists of characters
    true_list = list(true_string)
    predicted_list = list(predicted_string)
    # Compute hard accuracy using accuracy_score function
    hard_accuracy = accuracy_score(true_list, predicted_list)
    return hard_accuracy

def compute_all_hard_accuracies(true_sequences, predicted_sequences):
    num_seq = len(true_sequences)
    assert num_seq == len(predicted_sequences)
    accuracy_all = []
    accuracy_dict = {0:[], 50:[], 100:[], 150:[], 200:[], 250:[], 300:[], 350:[], 400:[], 450:[]}
    for i in range(num_seq):
        sequence = true_sequences[i]
        predicted_sequence = predicted_sequences[i]
        tmp_acc = compute_hard_accuracy(sequence, predicted_sequence)
        accuracy_all.append(tmp_acc)
        for j in range(0, len(sequence)-49, 50):
            tmp_acc = compute_hard_accuracy(sequence[j:j+50], predicted_sequence[j:j+50])
            accuracy_dict[j].append(float(tmp_acc))       
    result = sum(accuracy_all) / num_seq
    for key in accuracy_dict:
        if len(accuracy_dict[key]) <= 0:
            break
        accuracy_dict[key] = sum(accuracy_dict[key])/len(accuracy_dict[key])
    return result, accuracy_dict

def compute_soft_accuracies(true_sequences, predicted_sequences, threshold = 0):
    # soft accuracy takes into account partial matches or similarities 
    # between the predictions and the ground truth labels
    bl_matrix = bl.BLOSUM(62)
    num_seq = len(true_sequences)
    assert num_seq == len(predicted_sequences)
    accuracy_all = []
    accuracy_dict = {0:[], 50:[], 100:[], 150:[], 200:[], 250:[], 300:[], 350:[], 400:[], 450:[]}
    for i in range(num_seq):
        sequence = true_sequences[i]
        predicted_sequence = predicted_sequences[i]
        tmp_acc = calculate_soft_accuracy(sequence, predicted_sequence, bl_matrix, threshold)
        accuracy_all.append(tmp_acc)
        for j in range(0, len(sequence)-49, 50):
            tmp_acc = calculate_soft_accuracy(sequence[j:j+50], predicted_sequence[j:j+50], bl_matrix, threshold)
            accuracy_dict[j].append(tmp_acc) 
    result = sum(accuracy_all) / num_seq
    for key in accuracy_dict:
        if len(accuracy_dict[key]) <= 0:
            break
        accuracy_dict[key] = sum(accuracy_dict[key])/len(accuracy_dict[key])
    return result, accuracy_dict

def calculate_soft_accuracy(true_sequence, predicted_sequence, bl_matrix, threshold = 0):
    total_characters = len(true_sequence)
    assert total_characters == len(predicted_sequence)
    partial_matches = 0
    for true_aa, predicted_aa in zip(true_sequence, predicted_sequence):
        similarity_score = bl_matrix[true_aa][predicted_aa]
        if similarity_score >= threshold:
            partial_matches += 1
    soft_accuracy = partial_matches / total_characters
    return soft_accuracy

def compute_perplexity(probs, indexes):    
    metric = Perplexity()
    input = probs
    target = indexes
    metric.update(input, target)
    perplexity = metric.compute()
    return perplexity

def compute_perplexities(probs_all, indexes_all):
    num_seq = len(indexes_all)
    try:
        indexes_all[0][0][0]
    except TypeError: 
        indexes = torch.tensor(indexes_all)
        probs = torch.tensor(probs_all)
        tmp_perplexity = compute_perplexity(probs, indexes)
        return tmp_perplexity
    assert num_seq == len(probs_all)
    perplexity_all = []
    perplexity_dict = {0:[], 50:[], 100:[], 150:[], 200:[], 250:[], 300:[], 350:[], 400:[], 450:[]}
    for i in range(num_seq):
        indexes = torch.tensor(indexes_all[i])
        probs = torch.tensor(probs_all[i])
        tmp_perplexity = compute_perplexity(probs, indexes)
        perplexity_all.append(tmp_perplexity)
        for j in range(0, len(indexes)-49, 50):
            tmp_acc = compute_perplexity(probs[j:j+50], indexes[j:j+50])
            perplexity_dict[j].append(tmp_acc)  
    result = sum(perplexity_all) / num_seq
    for key in perplexity_dict:
        if len(perplexity_dict[key]) <= 0:
            break
        perplexity_dict[key] = float(sum(perplexity_dict[key])/len(perplexity_dict[key]))
    return float(result), perplexity_dict

In [87]:
import pickle
# paths to the saved .p files
query = "PF16754"
predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
true_data_file = os.path.join(data_dir, "true_data_" + query + ".p")
indexes_all_data_file = os.path.join(data_dir, "true_tokens_index_data_" + query + ".p")
probs_all_data_file = os.path.join(data_dir, "tokens_probs_data_" + query + ".p")

# Reload predicted_data from the .p file
predicted_data = False
with open(predicted_data_file, "rb") as file:
    predicted_data = pickle.load(file)

# Reload true_data from the .p file
true_data = False
with open(true_data_file, "rb") as file:
    true_data = pickle.load(file)
    
# Reload probs_all_data from the .p file
probs_all_data = False
with open(probs_all_data_file, "rb") as file:
    probs_all_data = pickle.load(file)
    
# Reload indexes_all_data from the .p file
indexes_all_data = False
with open(indexes_all_data_file, "rb") as file:
    indexes_all_data = pickle.load(file)

# Compute hard accuracy
accuracy, accuracy_dict = compute_all_hard_accuracies(true_data, predicted_data)

# Compute perplexity
perplexity, perplexity_dict = compute_perplexities(probs_all_data, indexes_all_data)

# Compute soft accuracy
soft_accuracy, soft_accuracy_dict = compute_soft_accuracies(true_data, predicted_data)

print("Accuracy:", accuracy, "\nSoft accuracy:", soft_accuracy, "\nPerplexity: ", perplexity)
print("Accuracy dict:", accuracy_dict, "\nSoft accuracy dict:", 
      soft_accuracy_dict, "\nPerplexity: ", perplexity_dict)
# print(true_data)
# print(predicted_data)

Accuracy: 0.10826210826210826 
Soft accuracy: 0.42165242165242167 
Perplexity:  25.882215758163895
Accuracy dict: {0: 0.12, 50: 0.06, 100: 0.16, 150: 0.14, 200: 0.08, 250: 0.1, 300: 0.1, 350: [], 400: [], 450: []} 
Soft accuracy dict: {0: 0.42, 50: 0.4, 100: 0.44, 150: 0.4, 200: 0.48, 250: 0.42, 300: 0.4, 350: [], 400: [], 450: []} 
Perplexity:  {0: 25.86396395982201, 50: 25.858447302513767, 100: 25.88541038509578, 150: 25.878199054664755, 200: 25.89244198426653, 250: 25.898550754591458, 300: 25.898527043770937, 350: [], 400: [], 450: []}


<h3>Code to make understand and debug</h3>

In [88]:
# DEVELOPMENT
# Example usage of perplexity computation
probs_all = [[[[0.3, 0.7]], [[0.6, 0.4]]], [[[0.2, 0.8]], [[0.5, 0.5]]]]
indexes_all = [[[0], [1]], [[1], [0]]]

perplexity = compute_perplexities(probs_all, indexes_all)
print("perplexity:", perplexity)


perplexity: (2.0563691990737216, {0: [], 50: [], 100: [], 150: [], 200: [], 250: [], 300: [], 350: [], 400: [], 450: []})


In [89]:
# DEVELOPMENT
# Example usage of hard and soft accuracy computation
true_sequences =      ['LS', 'LLPL', 'OOOOOOO'] # List of true sequences MITIMMNFHN
predicted_sequences = ['MS', 'OOOO', 'ASSSSSS']  # List of predicted sequences

# Compute hard accuracy
accuracy = compute_all_hard_accuracies(true_sequences, predicted_sequences)

# Compute soft accuracy
soft_accuracy = compute_soft_accuracies(true_sequences, predicted_sequences)

print("Accuracy:", accuracy, "\nSoft accuracy:", soft_accuracy)

Accuracy: (0.16666666666666666, {0: [], 50: [], 100: [], 150: [], 200: [], 250: [], 300: [], 350: [], 400: [], 450: []}) 
Soft accuracy: (0.3333333333333333, {0: [], 50: [], 100: [], 150: [], 200: [], 250: [], 300: [], 350: [], 400: [], 450: []})


<h3>Old Progen Generation Code:</h3>

In [ ]:
print(kw_to_name[9])
taxid = 9606 # homo sapiens taxonomy id from NCBI: https://www.ncbi.nlm.nih.gov/taxonomy
tax_lineage = taxa_to_lineage[taxid] # make lineage in ncbi ids
print(tax_lineage)
tax_lineage = [taxa_to_ctrl_idx[ite] for ite in tax_lineage] # now translated as ctrl code indices
print(tax_lineage)

kw_lineage = [677,9] # UniprotKB keywords from https://www.uniprot.org/docs/keywlist
print(kw_lineage)
kw_lineage = [kw_to_ctrl_idx[ite] for ite in kw_lineage] # now translated to ctrl code indices
print(kw_lineage)


example_seq = 'YMIQEEEWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIENIEEDFRNGLKLMLLLEVISGERLPKPDRGKMRFHKIANVNKALDYIASKGVKLVSIGAEEIVDGNVKMTLGMIWTIILRFAIQDISVEETSAKEGLLLWCQRKTAPYRNVNIQNFHTSWKDGLGLCALIHRHRPDLIDYSKLNKDDPIGNINLAMEIAEKHLDIPKMLDAEDIVNTPKPDERAIMTYVSCFYHAFAGAEQAETAANRICKVLAVNQENERLMEEYERLASELLEWIRRTIPWLENRTPAATMQAMQKKLEDFRDYRRKHKPPKVQEKCQLEINFNTLQTKLRISNRPAFMPSEGKMVSDIAGAWQRLEQAEKGYEEWLLNEIRRLERLEHLAEKFRQKASTHETWAYGKEQILLQKDYESASLTEVRALLRKHEAFESDLAAHQDRVEQIAAIAQELNELDYHDAVNVNDRCQKICDQWDRLGTLTQKRREALERMEKLLETIDQLHLEFAKRAAPFNNWMEGAMEDLQDMFIVHSIEEIQSLITAHEQFKATLPEADGERQSIMAIQNEVEKVIQSYNIRISSSNPYSTVTMDELRTKWDKVKQLVPIRDQSLQEELARQHANERLRRQFAAQANAIGPWIQNKMEEIARSSIQITGALEDQMNQLKQYEHNIINYKNNIDKLEGDHQLIQEALVFDNKHTNYTMEHIRVGWELLLTTIARTINEVETQILTRDAKGITQEQMNEFRASFNHFDRRKNGLMDHEDFRACLISMGYDLGEAEFARIMTLVDPNGQGTVTFQSFIDFMTRETADTDTAEQVIASFRILASDKPYILAEELRRELPPDQAQYCIKRMPAYSGPGSVPGALDYAAFSSALYGESDL'
prefix = example_seq[:3]
# prefix = ""
print("Prefix: ", prefix)
ref = example_seq[0:20]

print("Ref: ", ref)
penalty = 1.2
topk = 3

seed_seq = [aa_to_ctrl_idx[ii] for ii in prefix]
print("seed sequence: ", seed_seq)
# generate_num = len(kw_lineage+tax_lineage)+len(prefix+ref)
key_len = len(kw_lineage+tax_lineage)
generate_num = key_len +len(prefix+ref)
seq_length = min(generate_num, 511)

text = tax_lineage + kw_lineage + seed_seq
padded_text = text + [0] * (generate_num - len(text))
print(padded_text)
tokens_generated = np.tile(padded_text, (1,1))

#i = 1
for token in range(len(text)-1, generate_num-1):
    #print("counter: ", i)
    #i += 1
    print(tokens_generated[:, :seq_length].shape)
    prompt_logits = predict_fn(tokens_generated[:, :seq_length]).squeeze()
    _token = token if token < seq_length else -1
    if GPU:
        prompt_logits = prompt_logits.cpu().detach().numpy()
    else:
        prompt_logits = prompt_logits.gpu().detach().numpy()

    if penalty>0:
        penalized_so_far = set()
        # variable token_flag for first amminoacids (to count them if they are less that 4)
        if token >= key_len + 3:
            token_flag = 3  
        elif token - key_len - 3 <= 0:
            token_flag = 0
        else:
            token_flag = token
        #print(key_len)
        #print(token)
        #print(token_flag)
        for _ in range(token-token_flag,token+1):
            generated_token = tokens_generated[0][_] - (vocab_size-26) # added
            if generated_token in penalized_so_far:
                continue
            penalized_so_far.add(generated_token)
            prompt_logits[_token][generated_token] /= penalty

    # compute probabilities from logits
    prompt_probs = np.exp(prompt_logits[_token])
    prompt_probs = prompt_probs / sum(prompt_probs)
    pruned_list = np.argsort(prompt_probs)[::-1]

    if topk==1:
        idx = pruned_list[0]
    else:
        pruned_list = pruned_list[:topk]
        chosen_idx = torch.distributions.categorical.Categorical(logits=torch.tensor(np.expand_dims(prompt_logits[_token][pruned_list],0))).sample().numpy()[0]
        idx = pruned_list[chosen_idx]

    # assign the token for generation
    idx += (vocab_size-26) # added to convert 0 AA to original ctrl idx
    tokens_generated[0][token+1] = idx


tokens_generated_so_far = tokens_generated[0].squeeze()[:token+2]
tokens_generated_so_far = tokens_generated_so_far[(tokens_generated_so_far>=(vocab_size-26)) & (tokens_generated_so_far<(vocab_size-1))]
tokens_generated_so_far = ''.join([ctrl_idx_to_aa[c] for c in tokens_generated_so_far])

query = tokens_generated_so_far[len(seed_seq):]

print(prefix)
print(query)